In [11]:
from collections import Counter
import ConfigParser
import matplotlib.pyplot as plt
import networkx as nx
import sys
import time
from TwitterAPI import TwitterAPI
import pickle 
import re 
from collections import defaultdict
import datetime
import codecs
import json


In [12]:
# This method is done for you. Make sure to put your credentials in twitter.cfg.
def get_twitter(config_file):
    """ Read the config_file and construct an instance of TwitterAPI.
    Args:
      config_file ... A config file in ConfigParser format with Twitter credentials
    Returns:
      An instance of TwitterAPI.
    """
    config = ConfigParser.ConfigParser()
    config.read(config_file)
    twitter = TwitterAPI(
                   config.get('twitter', 'consumer_key'),
                   config.get('twitter', 'consumer_secret'),
                   config.get('twitter', 'access_token'),
                   config.get('twitter', 'access_token_secret'))
    return twitter

twitter = get_twitter('twitter.cfg')
print('Established Twitter connection.')

Established Twitter connection.


In [13]:
def robust_request(twitter, resource, params, max_tries=2):
    """ If a Twitter request fails, sleep for 15 minutes.
    Do this at most max_tries times before quitting.
    Args:
      twitter .... A TwitterAPI object.
      resource ... A resource string to request.
      params ..... A parameter dictionary for the request.
      max_tries .. The maximum number of tries to attempt.
    Returns:
      A TwitterResponse object, or None if failed.
    """
    for i in range(max_tries):
        request = twitter.request(resource, params)
        if request.status_code == 401:
            print 'Unauthorized data access. The user does not have access to such information'
            break
        elif "page does not exist" in request.text:
            print 'page does not exist'
            break
        elif request.status_code == 200:
            return request
        else:
            print >> sys.stderr, 'Got error:', request.text, '\nsleeping for 15 minutes.'
            sys.stderr.flush()
            time.sleep(61 * 15)
 

In [14]:
def get_followers_from_pickle(filename):
    """ Returns all followers of a Twitter account by retrieving saved data,
        using pickle method.
    
    Args: A string representing the screen_name of a Twitter account,
          whose followers we need.
    Returns : A dictionary of { screen_name : user_info } pairs of followers,
              where user_info is a dictionary object containing user information
    """
    fileObject = open(filename,'r')
    followers = pickle.load(fileObject)
    fileObject.close()
    return followers

# fname = 'streaming_user_tweets_2015-11-18 12:26:03.txt'
# data = get_followers_from_pickle(fname)
# print data
followers_list = get_followers_from_pickle('nu_followers.txt')
len(followers)

35984

In [13]:
# def get_followers():
#     followers = []
#     f = open ('iit_followers.txt','rb')
#     regexp = re.compile(r'p\d+')
#     for line in f:
#         if regexp.search(str(line)) is not None:
#             continue
#         else: 
#             if line.startswith('aV'):
#                 line = line[2:]
#                 followers.append(line.replace("\n",""))
#     followers.sort()
#     return followers

# followers_list = get_followers()

In [15]:
#This method collects tweets of all the followers of IIT's different twitter profiles and pickles them to a file
#Each file is time stamped and creates a new file at every 5000 tweets
#It takes the name of the output file and index in followers_list (list of screen_names of iit followers) 
#where one would like to start data collection. 
def print_tweets(outputfile,follower_index):
    tweets = 0
    followers = followers_list[follower_index:]
    f = codecs.open(outputfile,'a',encoding='utf8')
    for follower in followers:
        print "processing follower %s with index %d in followers_list" %(follower, followers_list.index(follower))
        request = robust_request(twitter,'statuses/user_timeline',{'screen_name': follower})
        if request is not None:
            if tweets <= 10000:
                for tweet in request:
                    if tweet is not None:
                        tweets += 1
                        json.dump(tweet,f)
                        f.write('\n')
                        print 'dumped tweet no %d' %(tweets)
            else:
                timestamp = datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')
                newfile = 'nu_outputfile_' + timestamp + '.txt'
                print 'calling new file name', newfile
                print_tweets(newfile,followers_list.index(follower))
    print 'End of collecting tweets of followers'
        

outputfile = 'nu_outputfile_'+ datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+ '.txt'
print_tweets(outputfile,0)


Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.


processing follower 005ia with index 0 in followers_list
dumped tweet no 1
dumped tweet no 2
dumped tweet no 3
dumped tweet no 4
dumped tweet no 5
dumped tweet no 6
dumped tweet no 7
dumped tweet no 8
dumped tweet no 9
dumped tweet no 10
dumped tweet no 11
processing follower 007MAV with index 1 in followers_list
dumped tweet no 12
dumped tweet no 13
dumped tweet no 14
dumped tweet no 15
dumped tweet no 16
dumped tweet no 17
dumped tweet no 18
dumped tweet no 19
dumped tweet no 20
dumped tweet no 21
dumped tweet no 22
dumped tweet no 23
dumped tweet no 24
dumped tweet no 25
dumped tweet no 26
dumped tweet no 27
dumped tweet no 28
dumped tweet no 29
dumped tweet no 30
dumped tweet no 31
processing follower 007kengom with index 2 in followers_list
dumped tweet no 32
dumped tweet no 33
dumped tweet no 34
dumped tweet no 35
dumped tweet no 36
dumped tweet no 37
dumped tweet no 38
dumped tweet no 39
dumped tweet no 40
dumped tweet no 41
dumped tweet no 42
dumped tweet no 43
dumped tweet no 

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



dumped tweet no 2017
dumped tweet no 2018
dumped tweet no 2019
dumped tweet no 2020
dumped tweet no 2021
dumped tweet no 2022
dumped tweet no 2023
dumped tweet no 2024
dumped tweet no 2025
dumped tweet no 2026
dumped tweet no 2027
dumped tweet no 2028
dumped tweet no 2029
dumped tweet no 2030
dumped tweet no 2031
dumped tweet no 2032
dumped tweet no 2033
dumped tweet no 2034
dumped tweet no 2035
processing follower 2854music with index 180 in followers_list
dumped tweet no 2036
dumped tweet no 2037
dumped tweet no 2038
dumped tweet no 2039
dumped tweet no 2040
dumped tweet no 2041
dumped tweet no 2042
dumped tweet no 2043
dumped tweet no 2044
dumped tweet no 2045
dumped tweet no 2046
dumped tweet no 2047
dumped tweet no 2048
dumped tweet no 2049
dumped tweet no 2050
dumped tweet no 2051
dumped tweet no 2052
dumped tweet no 2053
dumped tweet no 2054
dumped tweet no 2055
processing follower 28LCW with index 181 in followers_list
Unauthorized data access. The user does not have access to

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



dumped tweet no 4378
dumped tweet no 4379
dumped tweet no 4380
dumped tweet no 4381
dumped tweet no 4382
dumped tweet no 4383
dumped tweet no 4384
dumped tweet no 4385
dumped tweet no 4386
dumped tweet no 4387
dumped tweet no 4388
dumped tweet no 4389
dumped tweet no 4390
dumped tweet no 4391
dumped tweet no 4392
dumped tweet no 4393
dumped tweet no 4394
processing follower ABRAHAMLABRADA with index 360 in followers_list
dumped tweet no 4395
dumped tweet no 4396
dumped tweet no 4397
dumped tweet no 4398
dumped tweet no 4399
dumped tweet no 4400
dumped tweet no 4401
dumped tweet no 4402
dumped tweet no 4403
dumped tweet no 4404
dumped tweet no 4405
dumped tweet no 4406
dumped tweet no 4407
dumped tweet no 4408
dumped tweet no 4409
dumped tweet no 4410
dumped tweet no 4411
dumped tweet no 4412
dumped tweet no 4413
dumped tweet no 4414
processing follower ABSALLBZ with index 361 in followers_list
dumped tweet no 4415
dumped tweet no 4416
dumped tweet no 4417
dumped tweet no 4418
dumped t


dumped tweet no 5712
dumped tweet no 5713
dumped tweet no 5714
dumped tweet no 5715
dumped tweet no 5716
dumped tweet no 5717
dumped tweet no 5718
dumped tweet no 5719
dumped tweet no 5720
dumped tweet no 5721
dumped tweet no 5722
dumped tweet no 5723
dumped tweet no 5724
dumped tweet no 5725
dumped tweet no 5726
dumped tweet no 5727
dumped tweet no 5728
dumped tweet no 5729
dumped tweet no 5730
processing follower AIMBiomedical with index 441 in followers_list


TwitterConnectionError: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=5)

In [17]:
# followers_list.index('AIMBiomedical')
outputfile = 'nu_outputfile_'+ datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+ '.txt'
print_tweets(outputfile,441)


processing follower AIMBiomedical with index 441 in followers_list
dumped tweet no 1
dumped tweet no 2
dumped tweet no 3
dumped tweet no 4
dumped tweet no 5
dumped tweet no 6
dumped tweet no 7
dumped tweet no 8
dumped tweet no 9
dumped tweet no 10
dumped tweet no 11
dumped tweet no 12
dumped tweet no 13
dumped tweet no 14
dumped tweet no 15
dumped tweet no 16
dumped tweet no 17
dumped tweet no 18
dumped tweet no 19
dumped tweet no 20
processing follower AIM_Companies with index 442 in followers_list
dumped tweet no 21
dumped tweet no 22
dumped tweet no 23
dumped tweet no 24
dumped tweet no 25
dumped tweet no 26
dumped tweet no 27
dumped tweet no 28
dumped tweet no 29
dumped tweet no 30
dumped tweet no 31
dumped tweet no 32
dumped tweet no 33
dumped tweet no 34
dumped tweet no 35
dumped tweet no 36
dumped tweet no 37
dumped tweet no 38
dumped tweet no 39
dumped tweet no 40
processing follower AIPCareers with index 443 in followers_list
dumped tweet no 41
dumped tweet no 42
dumped tweet 

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



dumped tweet no 2763
dumped tweet no 2764
processing follower AbbeyChicago with index 621 in followers_list
dumped tweet no 2765
dumped tweet no 2766
dumped tweet no 2767
dumped tweet no 2768
dumped tweet no 2769
dumped tweet no 2770
dumped tweet no 2771
dumped tweet no 2772
dumped tweet no 2773
dumped tweet no 2774
dumped tweet no 2775
dumped tweet no 2776
dumped tweet no 2777
dumped tweet no 2778
dumped tweet no 2779
dumped tweet no 2780
dumped tweet no 2781
dumped tweet no 2782
dumped tweet no 2783
dumped tweet no 2784
processing follower AbbyLashmett with index 622 in followers_list
dumped tweet no 2785
dumped tweet no 2786
dumped tweet no 2787
dumped tweet no 2788
dumped tweet no 2789
dumped tweet no 2790
dumped tweet no 2791
dumped tweet no 2792
dumped tweet no 2793
dumped tweet no 2794
dumped tweet no 2795
dumped tweet no 2796
dumped tweet no 2797
dumped tweet no 2798
dumped tweet no 2799
dumped tweet no 2800
dumped tweet no 2801
dumped tweet no 2802
dumped tweet no 2803
dumped

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



dumped tweet no 5477
dumped tweet no 5478
dumped tweet no 5479
dumped tweet no 5480
dumped tweet no 5481
dumped tweet no 5482
dumped tweet no 5483
dumped tweet no 5484
dumped tweet no 5485
dumped tweet no 5486
dumped tweet no 5487
dumped tweet no 5488
dumped tweet no 5489
dumped tweet no 5490
dumped tweet no 5491
dumped tweet no 5492
dumped tweet no 5493
dumped tweet no 5494
dumped tweet no 5495
processing follower Agnes_Wasilewsk with index 801 in followers_list
dumped tweet no 5496
dumped tweet no 5497
dumped tweet no 5498
dumped tweet no 5499
dumped tweet no 5500
dumped tweet no 5501
dumped tweet no 5502
dumped tweet no 5503
dumped tweet no 5504
dumped tweet no 5505
dumped tweet no 5506
dumped tweet no 5507
dumped tweet no 5508
dumped tweet no 5509
dumped tweet no 5510
dumped tweet no 5511
dumped tweet no 5512
dumped tweet no 5513
dumped tweet no 5514
dumped tweet no 5515
processing follower AgusJupran with index 802 in followers_list
dumped tweet no 5516
dumped tweet no 5517
dumpe

...


processing follower CodeAbode_CHI with index 3677 in followers_list
dumped tweet no 7812
dumped tweet no 7813
dumped tweet no 7814
dumped tweet no 7815
dumped tweet no 7816
dumped tweet no 7817
dumped tweet no 7818
dumped tweet no 7819
dumped tweet no 7820
dumped tweet no 7821
dumped tweet no 7822
dumped tweet no 7823
dumped tweet no 7824
dumped tweet no 7825
dumped tweet no 7826
dumped tweet no 7827
dumped tweet no 7828
dumped tweet no 7829
dumped tweet no 7830
dumped tweet no 7831
processing follower CodyCejda with index 3678 in followers_list
dumped tweet no 7832
dumped tweet no 7833
dumped tweet no 7834
dumped tweet no 7835
dumped tweet no 7836
dumped tweet no 7837
dumped tweet no 7838
dumped tweet no 7839
dumped tweet no 7840
dumped tweet no 7841
dumped tweet no 7842
dumped tweet no 7843
dumped tweet no 7844
dumped tweet no 7845
dumped tweet no 7846
dumped tweet no 7847
dumped tweet no 7848
dumped tweet no 7849
dumped tweet no 7850
dumped tweet no 7851
processing follower CodyOrt

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



dumped tweet no 810
dumped tweet no 811
dumped tweet no 812
dumped tweet no 813
dumped tweet no 814
dumped tweet no 815
dumped tweet no 816
dumped tweet no 817
dumped tweet no 818
dumped tweet no 819
dumped tweet no 820
dumped tweet no 821
dumped tweet no 822
dumped tweet no 823
dumped tweet no 824
dumped tweet no 825
dumped tweet no 826
dumped tweet no 827
dumped tweet no 828
processing follower ContStrategy with index 3856 in followers_list
dumped tweet no 829
dumped tweet no 830
dumped tweet no 831
dumped tweet no 832
dumped tweet no 833
dumped tweet no 834
dumped tweet no 835
dumped tweet no 836
dumped tweet no 837
dumped tweet no 838
dumped tweet no 839
dumped tweet no 840
dumped tweet no 841
dumped tweet no 842
dumped tweet no 843
dumped tweet no 844
dumped tweet no 845
dumped tweet no 846
dumped tweet no 847
dumped tweet no 848
processing follower ContactHotelero with index 3857 in followers_list
dumped tweet no 849
dumped tweet no 850
dumped tweet no 851
dumped tweet no 852
du


dumped tweet no 1384
dumped tweet no 1385
dumped tweet no 1386
dumped tweet no 1387
dumped tweet no 1388
dumped tweet no 1389
dumped tweet no 1390
dumped tweet no 1391
processing follower CorrieAHarding with index 3891 in followers_list


TwitterConnectionError: HTTPSConnectionPool(host='api.twitter.com', port=443): Read timed out. (read timeout=5)

In [19]:
#followers_list.index('CorrieAHarding')
outputfile = 'nu_outputfile_'+ datetime.datetime.fromtimestamp(time.time()).strftime('%Y-%m-%d %H:%M:%S')+ '.txt'
print_tweets(outputfile,3981)

processing follower CrystalHess with index 3981 in followers_list
dumped tweet no 1
dumped tweet no 2
dumped tweet no 3
dumped tweet no 4
dumped tweet no 5
dumped tweet no 6
dumped tweet no 7
dumped tweet no 8
dumped tweet no 9
dumped tweet no 10
dumped tweet no 11
dumped tweet no 12
dumped tweet no 13
dumped tweet no 14
dumped tweet no 15
dumped tweet no 16
dumped tweet no 17
dumped tweet no 18
dumped tweet no 19
dumped tweet no 20
processing follower CrystalKWAN913 with index 3982 in followers_list
dumped tweet no 21
dumped tweet no 22
dumped tweet no 23
dumped tweet no 24
dumped tweet no 25
dumped tweet no 26
dumped tweet no 27
dumped tweet no 28
dumped tweet no 29
dumped tweet no 30
dumped tweet no 31
dumped tweet no 32
dumped tweet no 33
dumped tweet no 34
dumped tweet no 35
dumped tweet no 36
dumped tweet no 37
dumped tweet no 38
dumped tweet no 39
dumped tweet no 40
processing follower CrystalSJK with index 3983 in followers_list
dumped tweet no 41
dumped tweet no 42
dumped twee

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



dumped tweet no 2545
dumped tweet no 2546
dumped tweet no 2547
dumped tweet no 2548
dumped tweet no 2549
dumped tweet no 2550
dumped tweet no 2551
dumped tweet no 2552
dumped tweet no 2553
dumped tweet no 2554
dumped tweet no 2555
dumped tweet no 2556
dumped tweet no 2557
dumped tweet no 2558
dumped tweet no 2559
dumped tweet no 2560
dumped tweet no 2561
dumped tweet no 2562
dumped tweet no 2563
processing follower DThomasMooers with index 4161 in followers_list
dumped tweet no 2564
dumped tweet no 2565
dumped tweet no 2566
dumped tweet no 2567
dumped tweet no 2568
dumped tweet no 2569
dumped tweet no 2570
dumped tweet no 2571
dumped tweet no 2572
dumped tweet no 2573
dumped tweet no 2574
dumped tweet no 2575
dumped tweet no 2576
dumped tweet no 2577
dumped tweet no 2578
dumped tweet no 2579
dumped tweet no 2580
dumped tweet no 2581
dumped tweet no 2582
dumped tweet no 2583
processing follower DTiltz with index 4162 in followers_list
dumped tweet no 2584
dumped tweet no 2585
dumped tw

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



dumped tweet no 4736
dumped tweet no 4737
dumped tweet no 4738
dumped tweet no 4739
dumped tweet no 4740
dumped tweet no 4741
dumped tweet no 4742
dumped tweet no 4743
dumped tweet no 4744
dumped tweet no 4745
dumped tweet no 4746
dumped tweet no 4747
dumped tweet no 4748
dumped tweet no 4749
dumped tweet no 4750
dumped tweet no 4751
dumped tweet no 4752
dumped tweet no 4753
dumped tweet no 4754
processing follower DaschkoCo with index 4341 in followers_list
dumped tweet no 4755
dumped tweet no 4756
dumped tweet no 4757
dumped tweet no 4758
dumped tweet no 4759
dumped tweet no 4760
dumped tweet no 4761
dumped tweet no 4762
dumped tweet no 4763
dumped tweet no 4764
dumped tweet no 4765
dumped tweet no 4766
dumped tweet no 4767
dumped tweet no 4768
dumped tweet no 4769
dumped tweet no 4770
dumped tweet no 4771
dumped tweet no 4772
dumped tweet no 4773
dumped tweet no 4774
processing follower Dasmaninstitiut with index 4342 in followers_list
dumped tweet no 4775
dumped tweet no 4776
dump

...


processing follower Degree_Connect with index 4522 in followers_list
dumped tweet no 7289
dumped tweet no 7290
dumped tweet no 7291
dumped tweet no 7292
dumped tweet no 7293
dumped tweet no 7294
dumped tweet no 7295
dumped tweet no 7296
dumped tweet no 7297
dumped tweet no 7298
dumped tweet no 7299
dumped tweet no 7300
dumped tweet no 7301
dumped tweet no 7302
dumped tweet no 7303
dumped tweet no 7304
dumped tweet no 7305
dumped tweet no 7306
dumped tweet no 7307
dumped tweet no 7308
processing follower DelAidGhana with index 4523 in followers_list
dumped tweet no 7309
dumped tweet no 7310
dumped tweet no 7311
dumped tweet no 7312
dumped tweet no 7313
dumped tweet no 7314
dumped tweet no 7315
dumped tweet no 7316
dumped tweet no 7317
dumped tweet no 7318
dumped tweet no 7319
dumped tweet no 7320
dumped tweet no 7321
dumped tweet no 7322
dumped tweet no 7323
dumped tweet no 7324
dumped tweet no 7325
dumped tweet no 7326
dumped tweet no 7327
dumped tweet no 7328
processing follower Dela

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.



dumped tweet no 9716
dumped tweet no 9717
dumped tweet no 9718
dumped tweet no 9719
dumped tweet no 9720
dumped tweet no 9721
dumped tweet no 9722
dumped tweet no 9723
dumped tweet no 9724
dumped tweet no 9725
dumped tweet no 9726
dumped tweet no 9727
dumped tweet no 9728
dumped tweet no 9729
dumped tweet no 9730
dumped tweet no 9731
dumped tweet no 9732
dumped tweet no 9733
dumped tweet no 9734
processing follower DoFMedia with index 4701 in followers_list
dumped tweet no 9735
dumped tweet no 9736
dumped tweet no 9737
dumped tweet no 9738
dumped tweet no 9739
dumped tweet no 9740
dumped tweet no 9741
dumped tweet no 9742
dumped tweet no 9743
dumped tweet no 9744
dumped tweet no 9745
dumped tweet no 9746
dumped tweet no 9747
dumped tweet no 9748
dumped tweet no 9749
dumped tweet no 9750
dumped tweet no 9751
dumped tweet no 9752
dumped tweet no 9753
dumped tweet no 9754
processing follower DoHoangViet13 with index 4702 in followers_list
dumped tweet no 9755
dumped tweet no 9756
process

Got error: {"errors":[{"message":"Rate limit exceeded","code":88}]} 
sleeping for 15 minutes.


KeyboardInterrupt: 